# Level Up to Agentic AI: Roast My Resume Agent

### IEEE IGDTUW Workshop

**What you'll build:** An AI agent that reads your resume bullet points, roasts the weak ones, rewrites them stronger, and then *critiques its own rewrite* to make them perfect.

**3 Levels:**
- **Level 1** — Teach Your AI to Think (Structured Output)
- **Level 2** — Give Your AI Superpowers (Tool Use)
- **Level 3** — Make Your AI Argue With Itself (Self-Correction)

**+ Bonus:** Deploy it as a web app, ATS scoring, LinkedIn post generator

---

## Setup

**Before running this cell:**
1. Go to [aistudio.google.com](https://aistudio.google.com)
2. Click **"Get API Key"** → **"Create API key in new project"**
3. Copy the key
4. In Colab, click the **key icon** (Secrets) in the left sidebar
5. Add a new secret: Name = `GEMINI_API_KEY`, Value = your key
6. Toggle the **notebook access** switch ON

Then run the cell below:

In [ ]:
# ============================================================
# SETUP — Run this cell first!
# ============================================================
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata
import json

# Configure the API
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
# Use gemini-2.5-flash (reliable free tier; 2.0-flash and 2.5-pro often have limit: 0 on new keys).
model = genai.GenerativeModel('gemini-2.5-flash')

# Quick test — if this prints a response, you're good!
test = model.generate_content("Say 'Setup complete!' and nothing else.")
print(f"[OK] {test.text.strip()}")
print("\nYou're ready to build your agent.")

### Check rate limit / quota (optional)

Google doesn't expose "remaining requests" via API. You can:
- **Dashboard:** Open the link below (same Google account as your API key) to see RPM/TPM/RPD usage.
- **Probe:** Run the next cell to make one test request — if it succeeds, you have quota; if 429, it will show "retry in Xs".

In [ ]:
# ============================================================
# CHECK QUOTA / RATE LIMIT (run in Colab)
# ============================================================
# Official dashboard (open in browser — same Google account as API key):
print("Rate limit dashboard: https://ai.dev/rate-limit")
print("Or: https://aistudio.google.com/usage?tab=rate-limit\n")

# Probe: one request to see if you have quota right now
import re
try:
    r = model.generate_content("Reply with one word: OK")
    print("[OK] You have quota. Response:", r.text.strip()[:50])
except Exception as e:
    err = str(e)
    print("[429 / quota] Request failed.")
    if "retry in" in err.lower():
        sec = re.search(r"retry in (\d+\.?\d*)s", err, re.I)
        if sec:
            print(f"  → Wait {float(sec.group(1)):.0f}s and try again.")
    if "limit: 0" in err:
        print("  → Project has 0 quota. See README: create key from aistudio.google.com, enable API, or try another account.")

---

## Sample Resume Bullets

Pick one to roast (or use your own!):

In [ ]:
# ============================================================
# SAMPLE RESUME BULLETS — Pick one or write your own!
# ============================================================

samples = [
    "Proficient in MS Office and team management",
    "Worked on various projects using Python",
    "Good communication skills",
    "Responsible for handling database operations",
    "Participated in college hackathon and won prize",
    "Familiar with machine learning concepts",
    "Did internship at a startup and learned many things",
    "Team player with leadership qualities",
]

# === PICK YOUR BULLET ===
# Change the index or write your own string:
resume_bullet = samples[0]

print(f"[NOTE] Your bullet: \"{resume_bullet}\"")
print("\nLet's roast it.")

---

## LEVEL 1: Teach Your AI to Think

Most people use LLMs like this: *"Write me something"* → get a wall of text.

Agents think differently: *"Return structured data"* → get JSON you can actually use in code.

### Your Task

Write a prompt that tells Gemini to analyze the resume bullet and return **JSON** with these fields:
- `score` — a rating from 1 to 10
- `roast` — a funny but honest critique (2-3 sentences)
- `rewrite` — an improved version of the bullet
- `keywords_missing` — a list of strong keywords that should be added

**Hints:**
- Tell the model to return "ONLY valid JSON"
- Describe the persona (e.g., "brutally honest resume reviewer")
- Include the resume bullet in the prompt using the variable

In [ ]:
# ============================================================
# LEVEL 1: Structured Roast
# ============================================================

# ======================= YOUR CODE ==========================
# TODO: Write your prompt below.
# The prompt should tell Gemini to:
#   1. Act as a brutally honest resume reviewer
#   2. Analyze the resume_bullet variable
#   3. Return ONLY valid JSON with: score, roast, rewrite, keywords_missing
#
# Example structure (fill in the actual prompt):

prompt = f"""

YOUR PROMPT HERE

"""

# ===================== END YOUR CODE ========================

# --- Don't modify below this line ---
response = model.generate_content(prompt)
text = response.text.strip()
# Clean up markdown code fences if the model adds them
if text.startswith('```'):
    text = text.split('\n', 1)[1]
if text.endswith('```'):
    text = text.rsplit('```', 1)[0]

result_1 = json.loads(text.strip())

# Pretty display
print(f"[TARGET] Score: {result_1['score']}/10")
print(f"\n[FIRE] Roast: {result_1['roast']}")
print(f"\n[SPARKLE] Rewrite: {result_1['rewrite']}")
print(f"\n[KEY] Missing Keywords: {', '.join(result_1['keywords_missing'])}")

### What just happened?

You told the LLM to return **structured data** instead of paragraphs. This is the foundation of every AI agent:
- The output is *parseable* — your code can read and use each field
- The output is *consistent* — same fields every time
- The output is *pipeable* — you can feed it into the next step automatically

**Agentic Pattern #1: Structured Output** [CHECK]

---

## LEVEL 2: Give Your AI Superpowers

Right now, the agent is guessing what a "good" resume looks like. But what if it *knew* exactly what recruiters want for a specific role?

That's what **tools** do. A tool is just a Python function the agent can use to get information it doesn't have.

### The Tool

Run the cell below — it defines a function that returns what recruiters actually look for:

In [ ]:
# ============================================================
# THE TOOL — Pre-built for you (just run this cell)
# ============================================================

def get_role_requirements(role: str) -> str:
    """
    Returns what recruiters look for in a specific role.
    This is your agent's 'superpower' — external knowledge
    that the LLM doesn't have on its own.
    """
    requirements = {
        "software engineer": (
            "Strong DSA fundamentals, system design, clean code practices, "
            "CI/CD experience, testing frameworks, specific language/framework "
            "expertise (not just 'Python'), quantified impact metrics (%, $, users), "
            "open source contributions"
        ),
        "data scientist": (
            "Statistical modeling, Python/R proficiency, SQL mastery, "
            "ML frameworks (scikit-learn, PyTorch), A/B testing design, "
            "business impact quantification, data pipeline experience, "
            "visualization skills"
        ),
        "product manager": (
            "User research methodology, metrics-driven decisions (DAU, retention, conversion), "
            "roadmap planning, stakeholder management, market analysis, A/B testing, PRD writing"
        ),
        "ml engineer": (
            "Model training and fine-tuning, MLOps (MLflow, Kubeflow), "
            "distributed training, model serving (TensorRT, ONNX), "
            "data pipeline engineering, experiment tracking, production deployment"
        ),
        "frontend developer": (
            "React/Vue/Angular expertise, performance optimization (Core Web Vitals), "
            "accessibility (WCAG), responsive design, state management, "
            "testing (Jest, Cypress), design system experience"
        ),
        "backend developer": (
            "API design (REST/GraphQL), database optimization (SQL + NoSQL), "
            "caching strategies, message queues, microservices architecture, "
            "monitoring/observability, load testing"
        ),
        "devops engineer": (
            "CI/CD pipeline design, IaC (Terraform, Pulumi), container orchestration (K8s), "
            "monitoring (Prometheus, Grafana), cloud platforms (AWS/GCP/Azure), "
            "security best practices, incident response"
        ),
    }

    role_lower = role.lower()
    for key, value in requirements.items():
        if key in role_lower:
            return value
    return (
        "Quantified achievements, specific technologies, measurable impact, "
        "action verbs, industry-specific keywords, problem-solving examples"
    )

# --- Test the tool ---
target_role = "Software Engineer"  # Change this to any role!
context = get_role_requirements(target_role)

print(f"[TOOL] Tool called: get_role_requirements('{target_role}')")
print(f"\n[BRIEFCASE] Recruiters look for: {context}")

### Your Task

Now write a prompt that **uses the tool's output** to make a smarter roast.

The prompt should:
1. Tell the agent it's a resume expert who *knows what recruiters want*
2. Include the recruiter requirements from the tool (the `context` variable)
3. Include the `target_role` so the agent tailors its advice
4. Return JSON with: `score`, `roast`, `rewrite`, `missing_for_role`

In [ ]:
# ============================================================
# LEVEL 2: Tool-Powered Roast
# ============================================================

# ======================= YOUR CODE ==========================
# TODO: Write your prompt below.
# You have access to:
#   - resume_bullet (the original bullet)
#   - target_role (e.g., "Software Engineer")
#   - context (what recruiters look for — from the tool)
#
# The prompt should use ALL THREE variables to make
# a smarter, role-specific roast.
#
# Return JSON with: score, roast, rewrite, missing_for_role

prompt_2 = f"""

YOUR PROMPT HERE

"""

# ===================== END YOUR CODE ========================

# --- Don't modify below this line ---
response_2 = model.generate_content(prompt_2)
text_2 = response_2.text.strip()
if text_2.startswith('```'):
    text_2 = text_2.split('\n', 1)[1]
if text_2.endswith('```'):
    text_2 = text_2.rsplit('```', 1)[0]

result_2 = json.loads(text_2.strip())

# Pretty display
print(f"[TARGET] Score: {result_2['score']}/10")
print(f"\n[FIRE] Roast: {result_2['roast']}")
print(f"\n[SPARKLE] Rewrite: {result_2['rewrite']}")
print(f"\n[BRIEFCASE] Missing for {target_role}: {result_2['missing_for_role']}")

### What just happened?

Your agent just used a **tool** — it called `get_role_requirements()` to get information it didn't have, then used that context to make a smarter decision.

This is exactly how production agents work:
- Agents call APIs, search databases, read files
- The tool gives the agent **context** it can't get from its training data
- The agent **decides** how to use that context

**Agentic Pattern #2: Tool Use** [CHECK]

---

## LEVEL 3: Make Your AI Argue With Itself

The rewrite from Level 2 is good, but is it *perfect*? Probably not.

The **Reflection Pattern** is the secret weapon of production AI agents. It works like this:

1. Agent produces an output (the rewrite)
2. A **second LLM call** critiques that output — "Find 3 problems"
3. The agent fixes its own mistakes and produces a **final version**

The agent is literally arguing with itself to get better.

### Your Task

Write a prompt that:
1. Takes the rewrite from Level 2 (`result_2['rewrite']`)
2. Acts as a **senior hiring manager** at a top company
3. Finds **exactly 3 specific problems** with the rewrite
4. Produces the **final, perfected version**
5. Returns JSON with: `problems` (list of 3), `final_version`, `confidence_score` (1-10)

In [ ]:
# ============================================================
# LEVEL 3: The Self-Critique Loop
# ============================================================

rewrite_from_level_2 = result_2['rewrite']

# ======================= YOUR CODE ==========================
# TODO: Write the critique prompt below.
# You have access to:
#   - rewrite_from_level_2 (the rewrite from Level 2)
#   - target_role (the role they're applying for)
#   - context (recruiter requirements from the tool)
#
# The prompt should:
#   1. Persona: senior hiring manager at a FAANG company
#   2. Find exactly 3 specific problems with the rewrite
#   3. Produce the final, perfected bullet point
#   4. Return JSON with: problems, final_version, confidence_score

prompt_3 = f"""

YOUR PROMPT HERE

"""

# ===================== END YOUR CODE ========================

# --- Don't modify below this line ---
response_3 = model.generate_content(prompt_3)
text_3 = response_3.text.strip()
if text_3.startswith('```'):
    text_3 = text_3.split('\n', 1)[1]
if text_3.endswith('```'):
    text_3 = text_3.rsplit('```', 1)[0]

result_3 = json.loads(text_3.strip())

# Pretty display
print("[THINKING] Self-Critique Results:")
print("\nProblems found:")
for i, problem in enumerate(result_3['problems'], 1):
    print(f"  {i}. {problem}")

print(f"\n[OK] Final Version: {result_3['final_version']}")
print(f"\n[STATS] Confidence: {result_3['confidence_score']}/10")

# === THE BEFORE / AFTER ===
print("\n" + "="*50)
print("[NOTE] BEFORE vs AFTER")
print("="*50)
print(f"\n[X] Before: \"{resume_bullet}\"")
print(f"\n[OK] After:  \"{result_3['final_version']}\"")
print("\n" + "="*50)

### What just happened?

Your agent just **reviewed and corrected its own work**. This is the Reflection Pattern:

```
Output (Level 2) → Critique → Fix → Final Output
```

This is the exact same pattern used in:
- **Google's Gemini** — internal review loops before responding
- **Devin** (the AI SDE) — writes code, runs tests, fixes bugs
- **AutoGPT** — plans, executes, evaluates, adjusts

You just built it from scratch.

**Agentic Pattern #3: Self-Correction** [CHECK]

---

## The Full Pipeline

Let's wrap everything into one function so you can roast any bullet in a single call:

In [ ]:
# ============================================================
# HELPER FUNCTIONS
# ============================================================

def stage_print(stage, icon, color, message):
    """Print a visually distinct stage header."""
    colors = {
        'cyan': '\033[96m',
        'red': '\033[91m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'magenta': '\033[95m',
        'white': '\033[97m',
        'reset': '\033[0m'
    }
    c = colors.get(color, colors['white'])
    r = colors['reset']
    print(f"\n{c}{'='*60}{r}")
    try:
        # Try printing with icon
        print(f"{c}{icon}  [ {stage} ]{r}")
    except UnicodeEncodeError:
        # Fallback if terminal doesn't support emoji
        print(f"{c}[ {stage} ]{r}")
    print(f"{c}{'='*60}{r}")
    time.sleep(0.5)
    print(f"\n{message}\n")


def thinking_dots(text, duration=1.5):
    """Simulate agent thinking."""
    import sys
    sys.stdout.write(f"\033[90m{text}")
    for _ in range(3):
        time.sleep(duration / 3)
        sys.stdout.write(".")
        sys.stdout.flush()
    print("\033[0m")


def get_role_requirements(role: str) -> str:
    """TOOL: Returns what recruiters look for in a specific role."""
    requirements = {
        "software engineer": "Strong DSA fundamentals, system design, clean code practices, CI/CD experience, testing frameworks, specific language/framework expertise (not just 'Python'), quantified impact metrics (%, $, users), open source contributions",
        "data scientist": "Statistical modeling, Python/R proficiency, SQL mastery, ML frameworks (scikit-learn, PyTorch), A/B testing design, business impact quantification, data pipeline experience, visualization skills",
        "product manager": "User research methodology, metrics-driven decisions (DAU, retention, conversion), roadmap planning, stakeholder management, market analysis, A/B testing, PRD writing",
        "ml engineer": "Model training and fine-tuning, MLOps (MLflow, Kubeflow), distributed training, model serving (TensorRT, ONNX), data pipeline engineering, experiment tracking, production deployment",
        "frontend developer": "React/Vue/Angular expertise, performance optimization (Core Web Vitals), accessibility (WCAG), responsive design, state management, testing (Jest, Cypress), design system experience",
        "backend developer": "API design (REST/GraphQL), database optimization (SQL + NoSQL), caching strategies, message queues, microservices architecture, monitoring/observability, load testing",
        "devops engineer": "CI/CD pipeline design, IaC (Terraform, Pulumi), container orchestration (K8s), monitoring (Prometheus, Grafana), cloud platforms (AWS/GCP/Azure), security best practices, incident response"
    }
    role_lower = role.lower()
    for key, value in requirements.items():
        if key in role_lower:
            return value
    return "Quantified achievements, specific technologies, measurable impact, action verbs, industry-specific keywords, problem-solving examples"


def call_llm(prompt, retries=4, delay=10):
    """Call Gemini and return parsed JSON with retry logic and rate-limit backoff."""
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            text = response.text.strip()
            if text.startswith('```'):
                text = text.split('\n', 1)[1]
            if text.endswith('```'):
                text = text.rsplit('```', 1)[0]
            return json.loads(text.strip())
        except Exception as e:
            err_str = str(e).lower()
            is_rate_limit = "429" in str(e) or "resource exhausted" in err_str or "quota" in err_str
            if is_rate_limit and attempt < retries - 1:
                print(f"\n\033[93mRate limit hit. Retrying in {delay}s... (attempt {attempt+1}/{retries})\033[0m")
                time.sleep(delay)
                delay = min(delay * 2, 65)  # Exponential backoff, cap at 65s
            else:
                if is_rate_limit:
                    print("\033[91mQuota exceeded. Use gemini-2.5-flash in Setup, or try again later.\033[0m")
                raise e


print("[OK] Helpers loaded.")

---

# BONUS LEVELS

You've completed the core agent. Now let's add superpowers.

---

## BONUS: ATS Score Gamification

How well would your bullet survive an Applicant Tracking System? Let's find out.

In [ ]:
# ============================================================
# ATS SCORE — Before vs After
# ============================================================

original_bullet = resume_bullet
final_bullet = result_3['final_version']  # From Level 3

ats_prompt = f"""You are an Applicant Tracking System (ATS) scanner.
Score these two resume bullets for ATS compatibility for a {target_role} role.

Key factors: action verbs, quantified metrics, relevant keywords,
specificity, industry standard terminology.

Recruiters for {target_role} look for: {context}

Bullet A (original): "{original_bullet}"
Bullet B (improved): "{final_bullet}"

Return ONLY valid JSON with:
- "original_ats_score": integer 1-100
- "final_ats_score": integer 1-100
- "improvement": percentage improvement as a string (e.g., "+65%")
- "ats_tips": list of 3 tips for even higher ATS compatibility
"""

response_ats = model.generate_content(ats_prompt)
text_ats = response_ats.text.strip()
if text_ats.startswith('```'):
    text_ats = text_ats.split('\n', 1)[1]
if text_ats.endswith('```'):
    text_ats = text_ats.rsplit('```', 1)[0]
ats_result = json.loads(text_ats.strip())

# Display
print("[ROBOT] ATS COMPATIBILITY SCAN")
print("=" * 40)
print(f"\n[X] Original: {ats_result['original_ats_score']}/100")
bar_orig = "\u2588" * (ats_result['original_ats_score'] // 5) + "\u2591" * (20 - ats_result['original_ats_score'] // 5)
print(f"   [{bar_orig}]")

print(f"\n[OK] Final:    {ats_result['final_ats_score']}/100")
bar_final = "\u2588" * (ats_result['final_ats_score'] // 5) + "\u2591" * (20 - ats_result['final_ats_score'] // 5)
print(f"   [{bar_final}]")

print(f"\n[CHART] Improvement: {ats_result['improvement']}")
print(f"\n[TIP] Tips for even higher scores:")
for tip in ats_result['ats_tips']:
    print(f"   \u2022 {tip}")

---

## SECRET LEVEL 4: Deploy as a Web App

Right now, your agent lives in a notebook. Let's turn it into a **live web app** with a public URL — in 5 lines of code.

In [ ]:
# ============================================================
# SECRET LEVEL 4: Deploy with Gradio
# ============================================================
!pip install -q gradio

import gradio as gr

def roast_app(bullet: str, role: str) -> str:
    """Wrapper for the Gradio interface."""
    try:
        result = roast_my_resume(bullet, role)
        output = f"""## Results\n
**Original:** \"{result['original']}\"\n
**Initial Score:** {result['initial_score']}/10\n
**Roast:** {result['roast']}\n
---\n
**Problems Found:**\n"""
        for i, p in enumerate(result['problems_found'], 1):
            output += f"{i}. {p}\n"
        output += f"""\n---\n
**Final Version:** \"{result['final_version']}\"\n
**Confidence:** {result['confidence']}/10
"""
        return output
    except Exception as e:
        return f"Error: {str(e)}. Try again!"


demo = gr.Interface(
    fn=roast_app,
    inputs=[
        gr.Textbox(label="Resume Bullet", placeholder="Paste your resume bullet here..."),
        gr.Dropdown(
            label="Target Role",
            choices=["Software Engineer", "Data Scientist", "Product Manager",
                     "ML Engineer", "Frontend Developer", "Backend Developer",
                     "DevOps Engineer"],
            value="Software Engineer"
        ),
    ],
    outputs=gr.Markdown(label="Agent Output"),
    title="[FIRE] Roast My Resume Agent",
    description="An AI agent that roasts your resume, rewrites it, and argues with itself to make it perfect.",
    examples=[
        ["Proficient in MS Office and team management", "Software Engineer"],
        ["Familiar with machine learning concepts", "ML Engineer"],
        ["Good communication skills", "Product Manager"],
    ],
)

demo.launch(share=True)  # share=True gives you a PUBLIC URL!

**You just deployed an AI application.** Copy the public URL and send it to anyone — they can use your agent from their phone.

---

## BONUS: LinkedIn Post Generator

You just built and deployed an AI agent. Let the world know.

In [ ]:
# ============================================================
# LINKEDIN POST GENERATOR
# ============================================================

linkedin_prompt = """You are a LinkedIn post writer for a college student who just
built their first AI agent at a workshop.

Write a professional but enthusiastic LinkedIn post (150-200 words) about:
- They attended the "Level Up to Agentic AI" workshop by IEEE IGDTUW
- Mentored by Ishan (AI Engineer, GDG Cloud New Delhi speaker)
- They built a "Roast My Resume" AI Agent from scratch in Python
- The agent uses 3 agentic patterns: Structured Output, Tool Calling, Self-Correction
- They deployed it as a live web app using Gradio
- They're excited about AI Engineering and Agentic AI

Make it sound genuine, not salesy. Include 3-4 relevant hashtags.
Don't use excessive emojis — keep it professional.

Return ONLY valid JSON with:
- "post": the full LinkedIn post text
- "hashtags": list of hashtags used
"""

response_li = model.generate_content(linkedin_prompt)
text_li = response_li.text.strip()
if text_li.startswith('```'):
    text_li = text_li.split('\n', 1)[1]
if text_li.endswith('```'):
    text_li = text_li.rsplit('```', 1)[0]
li_result = json.loads(text_li.strip())

print("[NOTE] Your LinkedIn Post (copy-paste ready):\n")
print("=" * 50)
print(li_result['post'])
print("=" * 50)
print("\n[CLIP] Just copy the text above and paste it into LinkedIn!")

---

## What You Built Today

| Pattern | What It Does | Where It's Used in Production |
|---------|-------------|------------------------------|
| **Structured Output** | LLM returns JSON, not prose | Every AI agent ever |
| **Tool Use** | Agent calls functions for external data | Google Search, API calls, DB queries |
| **Self-Correction** | Agent critiques and fixes its own output | Devin, AutoGPT, production pipelines |

### Where to Go Next

- **LangGraph** — Build multi-agent systems with complex orchestration
- **MCP (Model Context Protocol)** — Universal standard for agent tools
- **Vertex AI / Cloud Run** — Deploy agents to production 24/7
- **Google AI Studio** — Your API key is free forever. Keep building.

### You just built what companies pay ₹20L+ for. Keep building.

---

*Workshop by Ishan — AI Engineer | GDG Cloud New Delhi*